# Orbital Calculations Assignment 
## by Starodumova Polina

Ground station and imaging target are located at  55N latitude, 37E longitude.  
Reference time or start of modeling scenario time is  2021-02-01 00:00:00 UTC.

In [111]:
from pyorbital.orbital import Orbital
from pyorbital import tlefile

from datetime import datetime
from datetime import timedelta

import numpy as np

time_f = '%Y-%m-%d %H:%M:%S'

**Task 1**  
Satellite position at reference time:

In [112]:
orb = Orbital('TERRA', tle_file='terra.tle')
utc = datetime(2021, 2, 1, 0, 0)
lon, lat, alt = orb.get_lonlatalt(utc)

print('Longitude is {:.2f}'.format(lon))
print('Latitude is {:.2f}'.format(lat))
print('Altitude is {:.2f}'.format(alt))

Longitude is 170.44
Latitude is 57.59
Altitude is 711.33


In [113]:
pos, vel = orb.get_position(utc)

print('Cartesian coordinates:')
print()
print('Position vector is', pos)
print('Velosity vector is', vel)

Cartesian coordinates:

Position vector is [ 0.31524669 -0.50709861  0.9347007 ]
Velosity vector is [ 0.01444024 -0.05863389 -0.03659657]


In [114]:
az, el = orb.get_observer_look(utc, 37.0, 55.0, 0.0)

print('Topocentric coordinates:')
print()
print('Azimuth is {:.2f}'.format(az))
print('Elevation is {:.2f}'.format(el))

Topocentric coordinates:

Azimuth is 26.34
Elevation is -25.57


**Task 2**  
AOS and LOS events for first contact of the satellite with the ground station:

In [115]:
aos, los, max_e_t = orb.get_next_passes(utc, 24, 37.0, 55.0, 0, horizon=10)[0]

print('AOS is ', aos.strftime(time_f))
print('LOS is', los.strftime(time_f))

AOS is  2021-02-01 08:10:47
LOS is 2021-02-01 08:20:04


**Task 3**  
The first pass culmination event time and satellite elevation at the culmination point:

In [116]:
az, el = orb.get_observer_look(max_e_t, 37.0, 55.0, 0)

print('Maximum elevation time is', max_e_t.strftime(time_f))
print('Azimuth in first maximum elevation time is {:.2f}'.format(az))
print('Elevation in first maximum elevation time is {:.2f}'.format(el))

Maximum elevation time is 2021-02-01 08:15:27
Azimuth in first maximum elevation time is 99.69
Elevation in first maximum elevation time is 49.70


**Task 4**  

In [117]:
print('First 5 imaging possibilities schedule for the target:')
print('(imaging incidence angles should be less than 30 deg so max elevation angle should be 60 deg)')
print()

time_image = [t[1] for t in orb.get_next_passes(utc, 120, 37.0, 55.0, 0, horizon=60)]

for t in time_image:
    print(t.strftime(time_f))

First 5 imaging possibilities schedule for the target:
(imaging incidence angles should be less than 30 deg so max elevation angle should be 60 deg)

2021-02-01 19:29:05
2021-02-02 08:59:06
2021-02-03 19:17:00
2021-02-04 08:46:55
2021-02-05 19:04:40


**Task 5**  
Simulate TLE sets for constellation of 4 satellites flying in 4 equally spaced orbital planes using given TLE as a starting point. Calculate first 5 imaging events schedule for the simulated constellation. 

Creating new TLEs: adding 90 deg (360 deg / 4 sat) to RAAN (Right Ascension of the Ascending Node – line 2 field 4) and changing the checksum digit.

In [119]:
sat1 = Orbital('TERRA', tle_file='terra.tle')
sat2 = Orbital('TERRA', tle_file='terra2.tle')
sat3 = Orbital('TERRA', tle_file='terra3.tle')
sat4 = Orbital('TERRA', tle_file='terra4.tle')

time_range = 33 # hours

pass1 = sat1.get_next_passes(utc, time_range, 37.0, 55.0, 0, horizon=60)
pass2 = sat2.get_next_passes(utc, time_range, 37.0, 55.0, 0, horizon=60)
pass3 = sat3.get_next_passes(utc, time_range, 37.0, 55.0, 0, horizon=60)
pass4 = sat4.get_next_passes(utc, time_range, 37.0, 55.0, 0, horizon=60)

all_pass = pass1 + pass2 + pass3 + pass4
all_pass = [t[1] for t in all_pass] 
all_pass.sort()

print('TLE 2 - 3:')
f = open('terra2.tle','r')
print(*f)
f = open('terra3.tle','r')
print(*f)
f = open('terra4.tle','r')
print(*f)

print('First 5 imaging possibilities schedule:')
print()
for t in all_pass:
    print(t.strftime(time_f))
    
    

TLE 2 - 3:
TERRA
 1 25994U 99068A   21029.94985485  .00000066  00000-0  24640-4 0  9999
 2 25994  98.1912 196.8209 0001366  97.1343  76.4657 14.57123255123173

TERRA
 1 25994U 99068A   21029.94985485  .00000066  00000-0  24640-4 0  9999
 2 25994  98.1912 286.8209 0001366  97.1343  76.4657 14.57123255123173

TERRA
 1 25994U 99068A   21029.94985485  .00000066  00000-0  24640-4 0  9999
 2 25994  98.1912  16.8209 0001366  97.1343  76.4657 14.57123255123174

First 5 imaging possibilities schedule:

2021-02-01 01:21:35
2021-02-01 12:53:39
2021-02-01 14:51:32
2021-02-01 19:29:05
2021-02-02 07:01:35
